In [1]:
import requests
import pandas as pd
"github:https://github.com/nprasad2077/nbaStats"

"api base url: http://rest.nbaapi.com"

'api base url: http://rest.nbaapi.com'

In [2]:
url_playoffs_lebron="http://rest.nbaapi.com/api/PlayerDataTotalsPlayoffs/name/Lebron James"
url_playoffs_kobe="http://rest.nbaapi.com/api/PlayerDataTotalsPlayoffs/name/Kobe Bryant"
url_playoffs_jordan="http://rest.nbaapi.com/api/PlayerDataTotalsPlayoffs/name/Michael Jordan"

In [48]:
response4=requests.get(url_playoffs_lebron)
response5=requests.get(url_playoffs_kobe)
response6=requests.get(url_playoffs_jordan)
print(response4,response5,response6)

<Response [200]> <Response [200]> <Response [200]>


In [49]:
data_playoffs_lebron=response4.json()
data_playoffs_kobe=response5.json()
data_playoffs_jordan=response6.json()
allplayoffs=data_playoffs_lebron + data_playoffs_kobe +data_playoffs_jordan

In [69]:
dfplayoffs=pd.DataFrame(allplayoffs)
dfplayoffs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               37 non-null     int64  
 1   playerName       37 non-null     object 
 2   position         37 non-null     object 
 3   age              37 non-null     int64  
 4   games            37 non-null     int64  
 5   gamesStarted     37 non-null     int64  
 6   minutesPg        37 non-null     float64
 7   fieldGoals       37 non-null     int64  
 8   fieldAttempts    37 non-null     int64  
 9   fieldPercent     37 non-null     float64
 10  threeFg          37 non-null     int64  
 11  threeAttempts    37 non-null     int64  
 12  threePercent     37 non-null     float64
 13  twoFg            37 non-null     int64  
 14  twoAttempts      37 non-null     int64  
 15  twoPercent       37 non-null     float64
 16  effectFgPercent  37 non-null     float64
 17  ft               3

In [70]:
dfplayoffs. drop(['id','gamesStarted', 'minutesPg', 'effectFgPercent', 'offensiveRb','defensiveRb'],axis=1,inplace=True)

In [71]:
dfplayoffs["fieldpts"]=dfplayoffs['fieldAttempts']*dfplayoffs["fieldPercent"]
dfplayoffs["threepts"]=dfplayoffs['threeAttempts']*dfplayoffs["threePercent"]
dfplayoffs["twopts"]=dfplayoffs['twoAttempts']*dfplayoffs["twoPercent"]
dfplayoffs["ftpts"]=dfplayoffs['ftAttempts']*dfplayoffs["ftPercent"]

## checks

In [72]:
lebron_check= dfplayoffs[dfplayoffs['playerName']=='LeBron James']
lebron_check["season"].value_counts().sum()

17

In [73]:
jordan_check= dfplayoffs[dfplayoffs['playerName']=='Kobe Bryant*']
jordan_check["season"].value_counts().sum()

15

In [74]:
jordan_check= dfplayoffs[dfplayoffs['playerName']=='Michael Jordan*']
jordan_check["season"].value_counts()
#there's data missing from Michael Jordan

1998    1
1997    1
1996    1
1995    1
1993    1
Name: season, dtype: int64

## Scrapped data from ESPN

In [75]:
#I scrapped the missing data from ESPN, see "SCrapping folder (Selenium) to see the process"
scrapped_playoffs_jordan=pd.read_csv(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\Scrapping\Selenium\jordan_playoffs")
scrapped_playoffs_jordan.drop(["Unnamed: 0"],axis=1,inplace=True)
scrapped_playoffs_jordan=scrapped_playoffs_jordan.iloc[:8]

In [76]:
scrapped_playoffs_jordan['playerName']="Michael Jordan*"
scrapped_playoffs_jordan['position']="SG"
scrapped_playoffs_jordan.rename(columns={
    'SEASON': 'season',
    'TEAM': 'team',
    'REB': 'totalRb',
    'AST': 'assists',
    'STL': 'steals',
    'BLK': 'blocks',
    '3P%': 'threePercent',
    'PF': 'personalFouls',
    'TO': 'turnovers',
    'PTS': 'points',
    'FG': 'fieldGoalsattempts&points',
    'FG%': 'fieldPercent',
    'FT': 'ftAttempts&points',  
    'FT%': 'ftPercent',
    '3PT': 'threeattempts&pts'
    
}, inplace=True)


In [77]:
scrapped_playoffs_jordan[['threepts','threeAttempts']] = scrapped_playoffs_jordan['threeattempts&pts'].str.split('-', expand=True)
scrapped_playoffs_jordan[['fieldpts','fieldGoals']]=scrapped_playoffs_jordan['fieldGoalsattempts&points'].str.split("-",expand=True)
scrapped_playoffs_jordan[['ftpts','ftAttempts']]=scrapped_playoffs_jordan['ftAttempts&points'].str.split("-",expand=True)

In [78]:
scrapped_playoffs_jordan[['threeAttempts', 'threepts', 
                         'fieldGoals', 'fieldpts', 'ftAttempts', 'ftpts']] = \
scrapped_playoffs_jordan[['threeAttempts', 'threepts', 
                         'fieldGoals', 'fieldpts', 'ftAttempts', 'ftpts']].astype(int)

In [79]:
scrapped_playoffs_jordan['twopts']=scrapped_playoffs_jordan['fieldpts']-scrapped_playoffs_jordan['threepts']
scrapped_playoffs_jordan['twoAttempts']=scrapped_playoffs_jordan['fieldGoals']-scrapped_playoffs_jordan['threeAttempts']

In [80]:
scrapped_playoffs_jordan.drop(["fieldGoalsattempts&points","threeattempts&pts","ftAttempts&points","OR","DR"],axis=1,inplace=True)


In [81]:
dfplayoffs.drop(['fieldAttempts', 'threeFg', 'twoFg', 'ft', 'playerId',"twoPercent"],axis=1,inplace=True)

In [82]:
scrapped_playoffs_jordan["season"]=scrapped_playoffs_jordan["season"].astype(int)

In [93]:
#Jordan's age is missing in ESPN's data, we know Jordan was born in 1964, so I calculated it:
def Age(i):
    
    return i -1964


scrapped_playoffs_jordan["age"]=scrapped_playoffs_jordan["season"].apply(Age)
#Playoff games per season is missing, I found them in another website, let's add this:


In [94]:
games=pd.read_csv(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\APIs\API_sports\datamichaeljordanplayoffs8592.csv",sep=";")

scrapped_playoffs_jordan=scrapped_playoffs_jordan.merge(games,on="season",how="left")

In [95]:
scrapped_playoffs_jordan.columns

Index(['season', 'team', 'fieldPercent', 'threePercent', 'ftPercent',
       'totalRb', 'assists', 'blocks', 'steals', 'personalFouls', 'turnovers',
       'points', 'playerName', 'position', 'threepts', 'threeAttempts',
       'fieldpts', 'fieldGoals', 'ftpts', 'ftAttempts', 'twopts',
       'twoAttempts', 'age', 'games'],
      dtype='object')

In [96]:
dfplayoffs.columns

Index(['playerName', 'position', 'age', 'games', 'fieldGoals', 'fieldPercent',
       'threeAttempts', 'threePercent', 'twoAttempts', 'ftAttempts',
       'ftPercent', 'totalRb', 'assists', 'steals', 'blocks', 'turnovers',
       'personalFouls', 'points', 'team', 'season', 'fieldpts', 'threepts',
       'twopts', 'ftpts'],
      dtype='object')

In [97]:
dfplayoffstotals = pd.concat([dfplayoffs, scrapped_playoffs_jordan], ignore_index=True)
dfplayoffstotals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   playerName     45 non-null     object 
 1   position       45 non-null     object 
 2   age            45 non-null     int64  
 3   games          45 non-null     int64  
 4   fieldGoals     45 non-null     int64  
 5   fieldPercent   45 non-null     float64
 6   threeAttempts  45 non-null     int64  
 7   threePercent   45 non-null     float64
 8   twoAttempts    45 non-null     int64  
 9   ftAttempts     45 non-null     int64  
 10  ftPercent      45 non-null     float64
 11  totalRb        45 non-null     int64  
 12  assists        45 non-null     int64  
 13  steals         45 non-null     int64  
 14  blocks         45 non-null     int64  
 15  turnovers      45 non-null     int64  
 16  personalFouls  45 non-null     int64  
 17  points         45 non-null     int64  
 18  team        

In [135]:
dfplayoffstotals["season"].unique()

array([2023, 2021, 2020, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011,
       2010, 2009, 2008, 2007, 2006, 2024, 2004, 2003, 2002, 2001, 2000,
       1999, 1998, 1997, 1996, 1995, 1993, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992], dtype=int64)

In [98]:
dfplayoffstotals["playerName"] = dfplayoffstotals["playerName"].replace("Kobe Bryant*", "Kobe Bryant")
dfplayoffstotals["playerName"]= dfplayoffstotals["playerName"].replace("Michael Jordan*","Michael Jordan")

In [131]:
#Data does not include the NBA champions, I will add this info

teamchampion=pd.read_csv(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\APIs\API_sports\teamchampionperseason.csv",sep=";",header=1)

In [139]:
teamchampion.rename(columns={"Year":"season"},inplace=True)
teamchampion.rename(columns={"Champion":"team"},inplace=True)

In [151]:
teamchampion["team"]= teamchampion["team"].replace("Los Angeles Lakers","LAL") \
    .replace('Chicago Bulls',"CHI") \
    .replace('Cleveland Cavaliers',"CLE") \
    .replace("Miami Heat","MIA")



In [154]:
teamchampion2=teamchampion[["season","team"]]
teamchampion2["rings"]=1
merged_data=pd.merge(dfplayoffstotals, teamchampion2, on=["season","team"],how="left",indicator=True)

c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [179]:
merged_data["rings"]=merged_data["rings"].fillna(0)

In [188]:
merged_data["rings"]=merged_data["rings"].astype(float)

In [189]:
merged_data.to_csv("Nba playoffs season data",sep=";",decimal=",",index=False)

## Getting per game data

In [197]:
dfplayoffs_pergame=merged_data.copy()
dfplayoffs_pergame.columns

Index(['playerName', 'position', 'age', 'games', 'fieldGoals', 'fieldPercent',
       'threeAttempts', 'threePercent', 'twoAttempts', 'ftAttempts',
       'ftPercent', 'totalRb', 'assists', 'steals', 'blocks', 'turnovers',
       'personalFouls', 'points', 'team', 'season', 'fieldpts', 'threepts',
       'twopts', 'ftpts', 'rings', '_merge'],
      dtype='object')

In [198]:
dfplayoffs_pergame["Pts per game"]=dfplayoffs_pergame["points"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["totalRb per game"]=dfplayoffs_pergame["totalRb"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["assists per game"]=dfplayoffs_pergame["assists"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["steals per game"]=dfplayoffs_pergame["steals"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["blocks per game"]=dfplayoffs_pergame["blocks"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["turnovers per game"]=dfplayoffs_pergame["turnovers"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["personalFouls per game"]=dfplayoffs_pergame["personalFouls"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["fieldpts per game"]=dfplayoffs_pergame["fieldpts"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["threepts per game"]=dfplayoffs_pergame["threepts"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["twopts per game"]=dfplayoffs_pergame["twopts"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)
dfplayoffs_pergame["ftpts per game"]=dfplayoffs_pergame["ftpts"].div(dfplayoffs_pergame['games'].replace(0, 1), axis=0)

In [199]:
dfplayoffs_pergame.drop(['totalRb', 'assists', 'steals', 'blocks', 'turnovers',
       'personalFouls', 'points', 'fieldpts', 'threepts',
       'twopts', 'ftpts','_merge'],axis=1,inplace=True)

In [200]:
dfplayoffs_pergame.to_csv("NBA playoffs pergame stats",sep=";",decimal=",",index=False)

In [202]:
dfplayoffs_pergame.columns

Index(['playerName', 'position', 'age', 'games', 'fieldGoals', 'fieldPercent',
       'threeAttempts', 'threePercent', 'twoAttempts', 'ftAttempts',
       'ftPercent', 'team', 'season', 'rings', 'Pts per game',
       'totalRb per game', 'assists per game', 'steals per game',
       'blocks per game', 'turnovers per game', 'personalFouls per game',
       'fieldpts per game', 'threepts per game', 'twopts per game',
       'ftpts per game'],
      dtype='object')